Initial notebook for teaching purposes with SQL + PyDough

This notebook uses TPC-H schema metadata and SQLite database connection for all examples.

We will now do the setup steps to load PyDough and its Jupyter extension:

In [125]:
%load_ext pydough.jupyter_extensions

import pydough

%load_ext pydough.jupyter_extensions

#Necessary for comparison
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal
import re
import dfcompare

import collections
import numpy as np
import sqlite3 as sql
import os

pd.options.display.float_format = '{:.6f}'.format

The pydough.jupyter_extensions extension is already loaded. To reload it, use:
  %reload_ext pydough.jupyter_extensions
The pydough.jupyter_extensions extension is already loaded. To reload it, use:
  %reload_ext pydough.jupyter_extensions


In [ ]:
#YOUR .SQL FILE TO CREATE THE DATABASE, COPY IT TO THIS FOLDER.
SQL_filename = 'ewallet.sql'

#METADATA FOR THE GRAPH .JSON
metadata_path = "./ewallet_graphs.json"
#GRAPH NAME
graph_name = "Ewallet"

#DESIRED DATABASE NAME
DB_name = "Ewallet.db"



with open(SQL_filename, 'r') as sql_file:
    sql_script = sql_file.read()

os.remove(DB_name)
connection = sql.connect(DB_name)
cursor = connection.cursor()
cursor.executescript(sql_script)

pydough.active_session.load_metadata_graph(metadata_path, graph_name)
pydough.active_session.connect_database("sqlite", database=DB_name)

DatabaseContext(connection=<pydough.database_connectors.database_connector.DatabaseConnection object at 0x7ff913fa67e0>, dialect=<DatabaseDialect.SQLITE: 'sqlite'>)

In [90]:
graph = pydough.active_session.metadata

print(pydough.explain(graph, verbose=True))

PyDough graph: Ewallet
Collections:
  Coupons
  MerchantBalances
  Merchants
  Notifications
  Transactions
  UserBalances
  UserSessions
  UserSettingSnapshots
  Users
Call pydough.explain(graph[collection_name]) to learn more about any of these collections.
Call pydough.explain_structure(graph) to see how all of the collections in the graph are connected.


In [5]:
graph = pydough.active_session.metadata

print(pydough.explain_structure(graph))

Structure of PyDough graph: Ewallet

  Coupons
  ├── cid
  ├── code
  ├── created_at
  ├── description
  ├── discount_type
  ├── discount_value
  ├── end_date
  ├── max_discount_amount
  ├── merchant_id
  ├── min_purchase_amount
  ├── redemption_limit
  ├── start_date
  ├── status
  ├── updated_at
  ├── merchant [one member of Merchants] (reverse of Merchants.coupons)
  └── transaction_used_in [multiple Transactions] (reverse of Transactions.coupon)

  MerchantBalances
  ├── balance
  ├── merchant_id
  ├── updated_at
  └── merchant [one member of Merchants] (reverse of Merchants.balances)

  Merchants
  ├── address
  ├── category
  ├── city
  ├── contact_email
  ├── contact_name
  ├── contact_phone
  ├── country
  ├── created_at
  ├── description
  ├── logo_url
  ├── mcc
  ├── mid
  ├── name
  ├── postal_code
  ├── state
  ├── status
  ├── sub_category
  ├── website_url
  ├── balances [multiple MerchantBalances] (reverse of MerchantBalances.merchant)
  ├── coupons [multiple Coupons] (r

# Pydough: Ewallet Queries.

## SQLite BasicQuery 

### How many distinct active users sent money per month in 2023? Return the number of active users per month (as a date), starting from the earliest date. Do not include merchants in the query. Only include successful transactions.

In [129]:
query = '''
SELECT strftime('%Y-%m', t.created_at) AS month, 
COUNT(DISTINCT t.sender_id) AS active_users 
FROM wallet_transactions_daily AS t 
JOIN users AS u ON t.sender_id = u.uid 
WHERE t.sender_type = 0 AND t.status = 'success' 
AND u.status = 'active' AND t.created_at >= '2023-01-01' 
AND t.created_at < '2024-01-01' 
GROUP BY month ORDER BY month;
'''
sql_output = pd.read_sql_query(query, connection)
sql_output

,month,active_users
0,2023-06,4
1,2023-07,1


In [130]:
%%pydough

selected_transactions= Transactions.WHERE((status=='success') & (created_at >= "2023-01-01") 
& (created_at < "2024-01-01") & (sending_user.status=='active') & (sender_type==0))(new_date= JOIN_STRINGS("-",YEAR(created_at),MONTH(created_at)))

output= PARTITION(selected_transactions, name="t", by=new_date)(
    month= new_date,
    active_users= NDISTINCT(t.sender_id)
)


#pydough.to_sql(output)
pydough.to_df(output)

,month,active_users
0,2023-6,4
1,2023-7,1


In [131]:
pydough_output = pydough.to_df(output)

dfcompare.compare_df(pydough_output, sql_output, query_category="a", question="a")

np.False_

In [29]:
%%pydough

selected_snapshot = UserSettingSnapshot((snapshop_date <= "2023-12-31") & (snapshop_date >= "2023-01-01")). TOP_K(1, by=snapshop_date.ASC())

selected_date = selected_snapshot.snapshop_date

output = UserSettingSnapshot.WHERE(snapshop_date == selected_date)(
avg_daily_limit=AVG(tx_limit_daily),
avg_monthly_limit=AVG(tx_limit_monthly))
#pydough.to_sql(output)
pydough.to_df(output)

PyDoughQDAGException: Unrecognized term of graph 'Ewallet': 'UserSettingSnapshot'

### Return merchants (merchant ID and name) who have not issued any coupons.

In [132]:
query = '''
SELECT m.mid AS merchant_id, m.name AS merchant_name 
FROM merchants AS m 
LEFT JOIN coupons AS c ON m.mid = c.merchant_id 
WHERE c.cid IS NULL;
'''
sql_output = pd.read_sql_query(query, connection)
sql_output

,merchant_id,merchant_name
0,11,FashionTrend
1,12,GreenGourmet
2,13,PetParadise
3,14,HomeTechSolutions
4,15,BookWorms


In [133]:
%%pydough

mercants_wihout_coupons = Merchants.WHERE(HASNOT(coupons))(merchan_id= mid, merchant_name= name)

#pydough.to_sql(output)
pydough.to_df(mercants_wihout_coupons)

,merchan_id,merchant_name
0,11,FashionTrend
1,12,GreenGourmet
2,13,PetParadise
3,14,HomeTechSolutions
4,15,BookWorms


In [134]:
pydough_output = pydough.to_df(mercants_wihout_coupons)

dfcompare.compare_df(pydough_output, sql_output, query_category="a", question="a")

True

### Return the distinct list of merchant IDs that have received money from a transaction. Consider all transaction types in the results you return, but only include the merchant ids in your final answer.




In [135]:
query = '''SELECT DISTINCT m.mid AS merchant_id FROM merchants AS m JOIN wallet_transactions_daily AS t ON m.mid = t.receiver_id WHERE t.receiver_type = 1;'''
sql_output = pd.read_sql_query(query, connection)
sql_output

,merchant_id
0,1
1,3
2,7
3,4
4,5
5,9
6,10
7,2


In [137]:
%%pydough
result = Merchants.WHERE(HAS(transactions_received.WHERE(receiver_type==1)))(merchant= mid)
pydough.to_df(result)

,merchant
0,1
1,2
2,3
3,4
4,5
5,7
6,9
7,10


In [138]:
pydough_output = pydough.to_df(result)

dfcompare.compare_df(pydough_output, sql_output, query_category="a", question="a")

np.True_

### Return the distinct list of user IDs who have received transaction notifications.

In [140]:
query = '''
SELECT DISTINCT user_id FROM notifications WHERE type = 'transaction';
'''
sql_output = pd.read_sql_query(query, connection)
sql_output

,user_id
0,1
1,2
2,3
3,4
4,10


In [141]:
%%pydough
users_with_notifications = Users.WHERE(HAS(notifications.WHERE(notification_type=='transaction')))(user_id= uid)
pydough.to_df(users_with_notifications)

,user_id
0,1
1,2
2,3
3,4
4,10


In [142]:
pydough_output = pydough.to_df(users_with_notifications)

dfcompare.compare_df(pydough_output, sql_output, query_category="a", question="a")

True

In [104]:
%%pydough

users_with_notifications = Notifications.WHERE(notification_type=='transaction')(user_id= user_id)

#pydough.to_sql(output)
pydough.to_df(users_with_notifications)

,user_id
0,1
1,2
2,3
3,4
4,10
5,1


### Return users (user ID and username) who have not received any notifications

In [143]:
query = '''
SELECT u.uid, u.username FROM users AS u 
LEFT JOIN notifications AS n ON u.uid = n.user_id 
WHERE n.id IS NULL;
'''
sql_output = pd.read_sql_query(query, connection)
sql_output

,uid,username
0,11,lisa_jones


In [144]:
%%pydough

users_without_notifications = Users.WHERE(HASNOT(notifications))(uid,username)

#pydough.to_sql(output)
pydough.to_df(users_without_notifications)

,uid,username
0,11,lisa_jones


In [145]:
pydough_output = pydough.to_df(users_without_notifications)

dfcompare.compare_df(pydough_output, sql_output, query_category="a", question="a")

True

### What are the top 2 most frequently used device types for user sessions and their respective counts?

In [146]:
query = '''
SELECT device_type, COUNT(*) AS COUNT 
FROM user_sessions 
GROUP BY device_type 
ORDER BY CASE WHEN COUNT IS NULL THEN 1 ELSE 0 END DESC, COUNT DESC LIMIT 2;
'''
sql_output = pd.read_sql_query(query, connection)
sql_output

,device_type,COUNT
0,web_app,15
1,mobile_app,8


In [147]:
%%pydough

selected_sessions= PARTITION(UserSessions, name="usession", by=device_type)(
    device_type=device_type,
    count= COUNT(usession)
)

sorted_sessions= selected_sessions.TOP_K(2,count.DESC())

#pydough.to_sql(output)
pydough.to_df(sorted_sessions)

,device_type,count
0,web_app,15
1,mobile_app,8


In [148]:
pydough_output = pydough.to_df(sorted_sessions)

dfcompare.compare_df(pydough_output, sql_output, query_category="a", question="a")

True

### What are the top 3 most common transaction statuses and their respective counts?

In [150]:
query = '''
SELECT status, COUNT(*) AS COUNT FROM wallet_transactions_daily 
GROUP BY status 
ORDER BY CASE WHEN COUNT IS NULL THEN 1 ELSE 0 END DESC, COUNT DESC LIMIT 3;
'''
df = pd.read_sql_query(query, connection)
df

,status,COUNT
0,success,19
1,pending,4
2,failed,2


In [149]:
%%pydough

transactions_by_status= PARTITION(Transactions, name="t", by=status)(
    status=status,
    count= COUNT(t)
)

sorted_transactions= transactions_by_status.TOP_K(3,count.DESC())

#pydough.to_sql(output)
pydough.to_df(sorted_transactions)

,status,count
0,success,19
1,pending,4
2,failed,2


In [151]:
pydough_output = pydough.to_df(sorted_transactions)

dfcompare.compare_df(pydough_output, df, query_category="a", question="a")

True

### What are the top 3 most frequently used coupon codes? Return the coupon code, total number of redemptions, and total amount redeemed.

In [152]:
query = '''
SELECT c.code AS coupon_code, COUNT(t.txid) AS redemption_count, SUM(t.amount) AS total_discount 
FROM coupons AS c JOIN wallet_transactions_daily AS t ON c.cid = t.coupon_id 
GROUP BY c.code 
ORDER BY CASE WHEN redemption_count IS NULL THEN 1 ELSE 0 END DESC, redemption_count DESC LIMIT 3;
'''
df = pd.read_sql_query(query, connection)
df

,coupon_code,redemption_count,total_discount
0,DINEDISCOUNT,2,52.750000
1,GLOWUP,2,125.000000
2,TECH20,2,36.000000


In [153]:
%%pydough

coupons_by_code= Coupons(coupon_code= code, redemption_count=COUNT(transaction_used_in.txid), total_discount=SUM(transaction_used_in.amount))

sorted_coupons= coupons_by_code.TOP_K(3,redemption_count.DESC())

#pydough.to_sql(output)
pydough.to_df(sorted_coupons)

,coupon_code,redemption_count,total_discount
0,TECH20,2,36.000000
1,DINEDISCOUNT,2,52.750000
2,GLOWUP,2,125.000000


In [154]:
pydough_output = pydough.to_df(sorted_coupons)

dfcompare.compare_df(pydough_output, df, query_category="a", question="a")

np.True_

### Which are the top 5 countries by total transaction amount sent by users, sender_type = 0? Return the country, number of distinct users who sent, and total transaction amount.

In [155]:
query = '''
SELECT u.country, COUNT(DISTINCT t.sender_id) AS user_count, SUM(t.amount) AS total_amount FROM users AS u 
JOIN wallet_transactions_daily AS t ON u.uid = t.sender_id 
WHERE t.sender_type = 0 
GROUP BY u.country 
ORDER BY CASE WHEN total_amount IS NULL THEN 1 ELSE 0 END DESC, total_amount DESC LIMIT 5;
'''
df = pd.read_sql_query(query, connection)
df

,country,user_count,total_amount
0,GB,1,2749.980000
1,CA,1,619.940000
2,US,1,344.980000
3,NZ,1,177.750000
4,DE,1,149.980000


In [156]:
%%pydough

transactions_by_sending_users= Transactions.WHERE(sender_type==0)(country= sending_user.country)

output= PARTITION(transactions_by_sending_users, name="t", by=country)(
    country=country,
    user_count= NDISTINCT(t.sender_id),
    total_amount= SUM(t.amount)
).TOP_K(5,total_amount.DESC())

#pydough.to_sql(output)
pydough.to_df(output)

,country,user_count,total_amount
0,GB,1,2749.980000
1,CA,1,619.940000
2,US,1,344.980000
3,NZ,1,177.750000
4,DE,1,149.980000


In [157]:
pydough_output = pydough.to_df(output)

dfcompare.compare_df(pydough_output, df, query_category="a", question="a")

True

### Who are the top 2 merchants (receiver type 1) by total transaction amount in the past 150 days (inclusive of 150 days ago)? Return the merchant name, total number of transactions, and total transaction amount.

In [158]:
query = '''
SELECT m.name AS merchant_name, COUNT(t.txid) AS total_transactions, SUM(t.amount) AS total_amount 
FROM merchants AS m 
JOIN wallet_transactions_daily AS t ON m.mid = t.receiver_id 
WHERE t.receiver_type = 1 AND t.created_at >= DATE('now', '-150 days') 
GROUP BY m.name 
ORDER BY total_amount DESC LIMIT 2;
'''
df = pd.read_sql_query(query, connection)
df


,merchant_name,total_transactions,total_amount
0,TechMart,2,399.980000
1,HandyCraft,1,199.990000


In [159]:
%%pydough

# Retrieve transactions where the receiver type is 1 (merchant) and the transaction date is after October 9th, 2024
selected_transactions = transactions_received.WHERE(
 (receiver_type == 1) & (created_at >= "2024-10-09")
)

merchant_transaction_summary=Merchants(
 merchant_name = name,
 total_transactions = COUNT(selected_transactions),
 total_amount = SUM(selected_transactions.amount),
).TOP_K(2, by=total_amount.DESC())
#pydough.to_sql(output)
pydough.to_df(merchant_transaction_summary)

,merchant_name,total_transactions,total_amount
0,TechMart,2,399.980000
1,HandyCraft,1,199.990000


In [160]:
pydough_output = pydough.to_df(merchant_transaction_summary)

dfcompare.compare_df(pydough_output, df, query_category="a", question="a")

True

In [ ]:
%%pydough

# Retrieve transactions where the receiver type is 1 (merchant) and the transaction date is after October 9th, 2024
transactions_from_sending_users = Transactions.WHERE(
    (receiver_type == 1) & (created_at >= "2024-10-09") 
)(merchant_name=receiving_merchant.name)  

# Partition the selected transactions by merchant name and calculate the total number of transactions and total amount for each merchant
merchant_transaction_summary = PARTITION(transactions_from_sending_users, name="transaction", by=m_name)(
    merchant_name=m_name,  
    total_transactions=COUNT(transaction), 
    total_amount=SUM(transaction.amount) 
).TOP_K(2, total_amount.DESC())  
#pydough.to_sql(output)
pydough.to_df(output)

,m_name,total_transactions,total_amount
0,TechMart,2,399.98
1,HandyCraft,1,199.99


### Give me today's median merchant wallet balance for all active merchants whose category contains 'retail'

In [49]:
query = '''
WITH retail_merchants AS (
    SELECT mid
    FROM merchants
    WHERE LOWER(category) LIKE LOWER('%retail%')
      AND status = 'active'
), 
merchant_balances AS (
    SELECT balance
    FROM wallet_merchant_balance_daily AS wmbd
    JOIN retail_merchants AS rm
        ON wmbd.merchant_id = rm.mid
    WHERE DATE(wmbd.updated_at) = date('now','-611 days')
)
SELECT AVG(balance) AS median_balance
FROM (
    SELECT balance
    FROM merchant_balances
    ORDER BY balance
    LIMIT 2 - (SELECT COUNT(*) FROM merchant_balances) % 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 FROM merchant_balances)
);
'''
df = pd.read_sql_query(query, connection)
df

,median_balance
0,164.485


In [ ]:
%%pydough

# Retrieve transactions where the receiver type is 1 (merchant) and the transaction date is after October 9th, 2024
selected_transactions = Merchants.WHERE(
 (CONTAINS(LOWER(category),'%retail%')) & (status == "active")
)

merchant_transaction_summary=Merchants(
 merchant_name = name,
 total_transactions = COUNT(selected_transactions),
 total_amount = SUM(selected_transactions.amount),
).TOP_K(2, by=total_amount.DESC())
#pydough.to_sql(output)
pydough.to_df(merchant_transaction_summary)

PyDoughQDAGException: Unrecognized term of simple table collection 'Merchants' in graph 'Ewallet': 'sending_merchant'

### What was the average transaction daily and monthly limit for the earliest setting snapshot in 2023?

In [118]:
query = '''
SELECT AVG(tx_limit_daily) AS avg_daily_limit, AVG(tx_limit_monthly) AS avg_monthly_limit 
FROM user_setting_snapshot 
WHERE snapshot_date = (SELECT MIN(snapshot_date) FROM user_setting_snapshot 
WHERE snapshot_date >= '2023-01-01' AND snapshot_date < '2024-01-01');
'''
df = pd.read_sql_query(query, connection)
df

,avg_daily_limit,avg_monthly_limit
0,501.0,1750.0


In [48]:
%%pydough

dates_range= (snapshot_date >= datetime.date(2023,1,1)) & (snapshot_date < datetime.date(2024,1,1))

min_snapshot_date = Ewallet(min_snapshot_date=MIN(UserSettingSnapshots.WHERE(dates_range).snapshot_date))

selected_snapshots= min_snapshot_date.UserSettingSnapshots.WHERE(snapshot_date == BACK(1).min_snapshot_date)

avg_transaction= PARTITION(selected_snapshots, name="par", by=(tx_limit_daily, tx_limit_monthly))(
avg_daily_limit=AVG(par.tx_limit_daily),
avg_monthly_limit=AVG(par.tx_limit_monthly))
#pydough.to_sql(output)
pydough.to_df(avg_transaction)

AssertionError: 

In [120]:
%%pydough

dates_range= (snapshot_date >= datetime.date(2023,1,1)) & (snapshot_date < datetime.date(2024,1,1))

min_snapshot_date = Ewallet(min_snapshot_date=MIN(UserSettingSnapshots.WHERE(dates_range).snapshot_date))

selected_snapshots= min_snapshot_date.UserSettingSnapshots.WHERE(snapshot_date == BACK(1).min_snapshot_date)

avg_transaction= min_snapshot_date(avg_daily_limit= AVG(UserSettingSnapshots.WHERE(snapshot_date == BACK(1).min_snapshot_date).tx_limit_daily))
#pydough.to_sql(output)
pydough.to_df(avg_transaction)

NotImplementedError: TODO: (gh #141) support BACK references that step from a child subtree back into a parent context.

### what was the average user session duration in seconds split by device_type?

In [161]:
query = '''
SELECT device_type, AVG(strftime('%s', session_end_ts) - strftime('%s', session_start_ts)) AS avg_session_duration_seconds 
FROM 
    user_sessions 
WHERE 
    session_end_ts IS NOT NULL 
GROUP BY 
    device_type;
'''
df = pd.read_sql_query(query, connection)
df

,device_type,avg_session_duration_seconds
0,mobile_app,1765.875000
1,web_app,1351.333333


In [162]:
%%pydough

has_session_end= PRESENT(session_end_ts)

selected_user_sessions = UserSessions.WHERE(has_session_end)

output=  PARTITION(selected_user_sessions, name="usession", by=device_type)(
    device_type=device_type,
    avg_session_duration_seconds= AVG(SECOND(usession.session_end_ts) - SECOND(usession.session_start_ts))
)
pydough.to_df(output)

,device_type,avg_session_duration_seconds
0,mobile_app,18.375000
1,web_app,3.333333


In [163]:
pydough_output = pydough.to_df(output)

dfcompare.compare_df(pydough_output, df, query_category="a", question="a")

np.False_

### Which merchants earliest coupon start date was within a year of the merchant's registration? Return the merchant id, registration date, and earliest coupon id and start date


In [19]:
query = '''
WITH earliest_coupons AS (
    SELECT c.merchant_id, 
           MIN(c.start_date) AS earliest_coupon_start_date 
    FROM coupons AS c
    GROUP BY c.merchant_id
)
SELECT m.mid AS merchant_id, 
       m.created_at AS merchant_registration_date, 
       ec.earliest_coupon_start_date, 
       c.cid AS earliest_coupon_id
FROM merchants AS m
JOIN earliest_coupons AS ec 
    ON m.mid = ec.merchant_id
JOIN coupons AS c 
    ON ec.merchant_id = c.merchant_id 
    AND ec.earliest_coupon_start_date = c.start_date
WHERE ec.earliest_coupon_start_date <= date(m.created_at, '+1 year');

'''
df = pd.read_sql_query(query, connection)
df

,merchant_id,merchant_registration_date,earliest_coupon_start_date,earliest_coupon_id
0,5,2022-06-20 00:00:00,2023-04-01,5
1,10,2023-02-01 00:00:00,2023-03-01,10


In [21]:
%%pydough
# Retrieve merchants with their earliest coupon start date, filtering those whose first coupon start date is before January 1st, 2023
merchants_with_earliest_coupons = Merchants(
    merchant_id=mid, 
    merchant_registration_date=created_at,  
    earliest_coupon_start_date=MIN(coupons.start_date) 
).WHERE(earliest_coupon_start_date <= "2023-01-01") 

# Retrieve coupon details for the merchants with the earliest coupon start dates
selected_earliest_coupons = merchants_with_earliest_coupons.coupons(
    merchant_id=BACK(1).merchant_id,  
    merchant_registration_date=BACK(1).merchant_registration_date, 
    earliest_coupon_start_date=BACK(1).earliest_coupon_start_date,  
    earliest_coupon_id=cid 
)

pydough.to_df(earliest_coupons)

,merchant_id,merchant_registration_date,earliest_coupon_start_date,earliest_coupon_id
0,2,2018-07-01 00:00:00,2023-01-01,2
1,8,2017-04-01 00:00:00,2022-12-01,8


### Which users did not get a notification within the first year of signing up? Return their usernames, emails and signup dates.


In [138]:
query = '''
SELECT u.username, u.email, u.created_at 
FROM users AS u 
LEFT JOIN notifications AS n ON u.uid = n.user_id 
AND n.created_at BETWEEN u.created_at 
AND '2025-01-01' 
WHERE n.user_id IS NULL;

'''
df = pd.read_sql_query(query, connection)
df

,username,email,created_at
0,john_doe,john.doe@email.com,2025-01-06 15:13:01
1,jane_smith,jane.smith@email.com,2024-12-06 15:13:01
2,huang2143,huang2143@example.com,2023-12-10 08:00:00
3,lisa_jones,lisa.jones@email.com,2023-09-05 15:20:00


In [23]:
%%pydough

users_without_notifications = Users.WHERE(HASNOT(notifications.WHERE((created_at >= user.created_at) & (created_at < "2025-01-01") )))(username, email)

#pydough.to_sql(output)
pydough.to_df(users_without_notifications)

,username,email
0,john_doe,john.doe@email.com
1,jane_smith,jane.smith@email.com
2,huang2143,huang2143@example.com
3,lisa_jones,lisa.jones@email.com


# Advance queries

### Calculate the CPUR for each merchant, considering only successful transactions. Return the merchant name and CPUR. CPUR (coupon usage rate) = number of distinct coupons used / number of distinct transactions.

In [62]:
query = '''
SELECT m.name, (COUNT(DISTINCT wtd.coupon_id) * 1.0 / NULLIF(COUNT(DISTINCT wtd.txid), 0)) AS CPUR 
FROM wallet_transactions_daily AS wtd 
JOIN merchants AS m ON wtd.receiver_id = m.mid 
WHERE wtd.status = 'success' 
GROUP BY m.name;
'''
df = pd.read_sql_query(query, connection)
df

,name,CPUR
0,BeautyTrending,0.333333
1,FitLifeGear,0.000000
2,GameRush,1.000000
3,HandyCraft,0.000000
4,TechMart,0.142857
5,UrbanDining,0.500000
6,ZenHomeGoods,1.000000


In [106]:
%%pydough

# Filter the transactions to get only successful ones
successful_transactions= transactions_received.WHERE(status=="success")

# Calculate the CPUR for the merchant
merchant_cpur_summary= Merchants.WHERE(HAS(successful_transactions))(
    name= name,
    CPUR= NDISTINCT(successful_transactions.coupon_id) * 1.0 / NDISTINCT(successful_transactions.txid)
)

pydough.to_df(merchant_cpur_summary)

,name,CPUR
0,TechMart,0.142857
1,FitLifeGear,0.000000
2,UrbanDining,0.500000
3,HandyCraft,0.000000
4,ZenHomeGoods,1.000000
5,BeautyTrending,0.333333
6,GameRush,1.000000


### For users in the US and Canada, how many total notifications were sent in each of the last 3 weeks excluding the current week? How many of those were sent on weekends?

In [73]:
query = '''
SELECT 
    date(n.created_at, '-' || ((strftime('%w', n.created_at) + 6) % 7) || ' days') AS WEEK, 
    COUNT(*) AS total_notifications, 
    COUNT(CASE 
        WHEN strftime('%w', n.created_at) IN ('0', '6') THEN 1 
    END) AS weekend_notifications 
FROM notifications AS n 
JOIN users AS u ON n.user_id = u.uid 
WHERE 
    u.country IN ('US', 'CA') 
    AND n.created_at >= date('now', '-' || ((strftime('%w', 'now') + 6) % 7) || ' days', '-21 days') 
    AND n.created_at < date('now', '-' || ((strftime('%w', 'now') + 6) % 7) || ' days') 
GROUP BY WEEK;

'''
df = pd.read_sql_query(query, connection)
df

,WEEK,total_notifications,weekend_notifications
0,2025-01-13,1,0
1,2025-01-20,1,0
2,2025-01-27,2,1


In [ ]:
%%pydough

end_date = DATETRUNC(DATE('now'), 'week')

start_date = start_date - Interval(days=21)

Notifications.WHERE(
 ISIN(user.country,('US', 'CA')) &
 (start_date <= created_at) & (start_date < end_date)
).PARTITION_BY(week = DATETRUNC(start_date, 'week'))(
 week,
 total_notifications = COUNT()
 weekend_notifications = COUNT(CLUSTER.WHERE(DAYOFWEEK(created_at) in (0, 6)))
)

### How many active retail merchants have issued coupons? Return the merchant name and the total number of coupons issued.

In [77]:
query = '''
SELECT 
    m.name, COUNT(c.cid) AS total_coupons 
FROM
    merchants AS m 
JOIN 
    coupons AS c ON m.mid = c.merchant_id 
WHERE 
    m.status = 'active' AND LOWER(m.category) LIKE '%retail%' 
GROUP BY m.name;
'''
df = pd.read_sql_query(query, connection)
df

,name,total_coupons
0,BeautyTrending,1
1,FitLifeGear,1
2,GameRush,1
3,HandyCraft,1
4,TechMart,1
5,ZenHomeGoods,1


In [4]:
%%pydough

# Retrieve merchant summary for active merchants in the "retail" category who have coupons
active_retail_merchants_with_coupons = Merchants.WHERE(
    (status == "active") & 
    (CONTAINS(LOWER(category), '%retail%')) &  
    HAS(coupons) 
)(
    merchant_name=name, 
    total_coupons=COUNT(coupons)  
)


pydough.to_df(active_retail_merchants_with_coupons)

,merchant_name,total_coupons
0,TechMart,1
1,FitLifeGear,1
2,HandyCraft,1
3,ZenHomeGoods,1
4,BeautyTrending,1
5,GameRush,1


### How many wallet transactions were made by users from the US in the last 7 days inclusive of today? Return the number of transactions and total transaction amount.

In [34]:
query = '''
SELECT 
    COUNT(*) AS num_transactions, 
    SUM(amount) AS total_amount 
FROM wallet_transactions_daily AS t 
JOIN users AS u ON t.sender_id = u.uid 
WHERE 
    u.country = 'US' 
    AND t.created_at >=  '2024-08-24'
    AND t.created_at < DATE('now', '+1 day'); 

'''
df = pd.read_sql_query(query, connection)
df

,num_transactions,total_amount
0,1,49.99


In [ ]:
%%pydough
# Define the start date for filtering transactions
start_date = DATE('now', '-7 days')
# Filter transactions based on the creation date and sending user's country
us_transactions = Transactions.WHERE(
    (created_at >= start_date) & 
    (sending_user.country == 'US') 
)
# Calculate the number of transactions and the total amount for the filtered transactions
transaction_summary = Ewallet(
    num_transactions=COUNT(us_transactions), 
    total_amount=SUM(us_transactions.amount)  
)

pydough.to_df(transaction_summary)


,num_transactions,total_amount
0,5,374.93


### What is the average AMB for user wallets updated in the past week, inclusive of 7 days ago? Return the average balance.


In [84]:
query = '''
SELECT 
    AVG(balance) AS AMB 
FROM wallet_user_balance_daily 
WHERE updated_at >= DATE('now', '-7 days'); 
'''
df = pd.read_sql_query(query, connection)
df

,AMB
0,848.26


In [89]:
%%pydough

d= DATE('now', '-7 days')
selected_user_balances= UserBalances.WHERE(updated_at >= "2025-02-01")

amb= Ewallet(AMB= AVG(selected_user_balances.balance))
pydough.to_df(amb)

,AMB
0,848.26


### What is the LUB for each user. LUB = Latest User Balance, which is the most recent balance for each user



In [90]:
query = '''
WITH user_balances AS (
    SELECT 
        user_id, 
        balance, 
        ROW_NUMBER() OVER (
            PARTITION BY user_id 
            ORDER BY 
                CASE WHEN updated_at IS NULL THEN 1 ELSE 0 END DESC, 
                updated_at DESC
        ) AS rn 
    FROM wallet_user_balance_daily
) 
SELECT 
    user_id, 
    balance 
FROM user_balances 
WHERE rn = 1;

'''
df = pd.read_sql_query(query, connection)
df

,user_id,balance
0,1,2733.92
1,2,155.24
2,3,2775.25
3,4,2500.00
4,5,155.99
5,6,29.95
6,7,172.98
7,8,0.00
8,9,125.00
9,10,219.98


In [ ]:
%%pydough

latest_balance = BEST(balances, by=updated_at.DESC())

Users(uid, latest_balance.balance)
pydough.to_df(amb)

### What is the marketing opt-in preference for each user? Return the user ID and boolean opt-in value

In [38]:
query = '''
WITH user_latest_setting AS (
    SELECT 
        u.uid, 
        s.marketing_opt_in, 
        s.created_at, 
        ROW_NUMBER() OVER (
            PARTITION BY u.uid 
            ORDER BY 
                CASE WHEN s.created_at IS NULL THEN 1 ELSE 0 END DESC, 
                s.created_at DESC
        ) AS rn
    FROM users AS u
    JOIN user_setting_snapshot AS s 
        ON u.uid = s.user_id
)
SELECT 
    UID, 
    marketing_opt_in 
FROM user_latest_setting 
WHERE rn = 1;


'''
df = pd.read_sql_query(query, connection)
df

,uid,marketing_opt_in
0,1,0
1,2,1
2,3,0
3,4,1
4,5,0
5,6,1
6,7,1
7,8,0
8,9,1
9,10,1


### What is the MRR for each merchant? Return the merchant name, category, revenue amount, and revenue rank. MRR = Merchant Revenue Rank, which ranks merchants based on amounts from successfully received transactions only. Filter receiver_type=1 in wallet_transactions_daily for merchants. Merchant with rank 1 has the highest revenue.

In [81]:
query1 = '''
WITH merchant_revenue AS (
    SELECT 
        m.mid, 
        m.category AS merchant_category, 
        SUM(w.amount) AS total_revenue 
    FROM merchants AS m
    INNER JOIN wallet_transactions_daily AS w 
        ON m.mid = w.receiver_id 
        AND w.receiver_type = 1
    WHERE w.status = 'success'
    GROUP BY m.mid, m.category
)
SELECT 
    *, 
    RANK() OVER (
        ORDER BY 
            CASE WHEN total_revenue IS NULL THEN 1 ELSE 0 END DESC, 
            total_revenue DESC
    ) AS mrr 
FROM merchant_revenue;
'''

query2 = '''
WITH merchant_revenue AS (
    SELECT 
        m.name, 
        m.category AS merchant_category, 
        SUM(w.amount) AS total_revenue 
    FROM merchants AS m
    INNER JOIN wallet_transactions_daily AS w 
        ON m.mid = w.receiver_id 
        AND w.receiver_type = 1
    WHERE w.status = 'success'
    GROUP BY m.name, m.category
)
SELECT 
    *, 
    RANK() OVER (
        ORDER BY 
            CASE WHEN total_revenue IS NULL THEN 1 ELSE 0 END DESC, 
            total_revenue DESC
    ) AS mrr 
FROM merchant_revenue;
'''

query3 = '''
WITH merchant_revenue AS (
    SELECT 
        m.mid, 
        m.name, 
        m.category AS merchant_category, 
        SUM(w.amount) AS total_revenue 
    FROM merchants AS m
    INNER JOIN wallet_transactions_daily AS w 
        ON m.mid = w.receiver_id 
        AND w.receiver_type = 1
    WHERE w.status = 'success'
    GROUP BY m.mid, m.name, m.category
)
SELECT 
    *, 
    RANK() OVER (
        ORDER BY 
            CASE WHEN total_revenue IS NULL THEN 1 ELSE 0 END DESC, 
            total_revenue DESC
    ) AS mrr 
FROM merchant_revenue;
'''

# Run each query separately
df1 = pd.read_sql_query(query1, connection)
df2 = pd.read_sql_query(query2, connection)
df3 = pd.read_sql_query(query3, connection)

# Optionally, you can concatenate the results into one DataFrame
df = pd.concat([df3], ignore_index=True)
df


,mid,name,merchant_category,total_revenue,mrr
0,1,TechMart,retail (hardware),435.98,1
1,9,BeautyTrending,Retail,125.00,2
2,3,UrbanDining,Food & Dining,109.99,3
3,2,FitLifeGear,retail (hardware),72.98,4
4,5,HandyCraft,Retail,55.99,5
5,10,GameRush,Retail,29.99,6
6,7,ZenHomeGoods,Retail,17.99,7


In [80]:
%%pydough

successful_transactions= transactions_received.WHERE((receiver_type == 1) & (status=="success"))

mrr_summary= Merchants.WHERE(HAS(successful_transactions))(
   total_revenue= SUM(successful_transactions.amount)
)(
   mid,
   category,
   name,
   total_revenue,
   mrr = RANKING(by=total_revenue.DESC(), levels=1)
)
pydough.to_df(mrr_summary)

/home/ara/miniconda3/envs/pydough-bodo/lib/python3.12/site-packages/pydough/sqlglot/sqlglot_relational_expression_visitor.py:88: UserWarning: PyDough when using SQLITE dialect does not support ascending ordering with nulls first (changed to nulls last)
  warnings.warn(


,mid,category,name,total_revenue,mrr
0,1,retail (hardware),TechMart,435.98,1
1,9,Retail,BeautyTrending,125.00,2
2,3,Food & Dining,UrbanDining,109.99,3
3,2,retail (hardware),FitLifeGear,72.98,4
4,5,Retail,HandyCraft,55.99,5
5,10,Retail,GameRush,29.99,6
6,7,Retail,ZenHomeGoods,17.99,7


### What is the PMDAU (Per Month Daily Active Users) for wallet transactions in the last 2 months excluding the current month? PMDAU (Per Month Daily Active Users) = COUNT(DISTINCT(sender_id) ... WHERE t.sender_type = 0. Truncate created_at to month for aggregation.

In [97]:
query = '''
SELECT 
    strftime('%Y-%m', t.created_at) AS month, 
    COUNT(DISTINCT t.sender_id) AS active_users 
FROM wallet_transactions_daily AS t 
WHERE 
    t.sender_type = 0 
    AND t.created_at >= date('now', 'start of month', '-2 months') 
    AND t.created_at < date('now', 'start of month') 
GROUP BY month;

'''
df = pd.read_sql_query(query, connection)
df

,month,active_users
0,2024-12,1
1,2025-01,1


In [96]:
%%pydough

# In PyDough we do not have a method DATE('now', '-6 months') to return the date.
# Then we take 2025-02-10 (current day when it was made) as "now" to calculate the date if we go 6 months in the past.
# This would be: "2024-12-01" as start date and '2025-02-01' as end_date 


# Define the start date (2 months before the start of the current month)
start_date = DATE('now', 'start of month', '-2 months')

# Define the end date (start of the current month)
end_date = DATE('now', 'start of month')

# Filter successful transactions based on sender type and creation date range
successful_transactions = Transactions.WHERE(
    (sender_type == 0) &  
    (created_at >= start_date) &  
    (created_at < end_date) 
)(year_month= JOIN_STRINGS("-",YEAR(created_at),MONTH(created_at)))

# Group transactions by month and calculate the number of distinct active users
pmdau_summary = PARTITION(successful_transactions, name="transc", by=year_month)(
    month=year_month,  
    active_users=NDISTINCT(transc.sender_id) 
)

pydough.to_df(pmdau_summary)

,month,active_users
0,2024-12,1
1,2025-1,1


### What is the total number of wallet transactions sent by each user that is not a merchant? Return the user ID and total transaction count.

In [98]:
query = '''
WITH user_transactions AS (
    SELECT u.uid, t.txid 
    FROM users AS u 
    JOIN wallet_transactions_daily AS t 
        ON u.uid = t.sender_id 
    WHERE t.sender_type = 0
)
SELECT 
    UID, 
    COUNT(txid) AS total_transactions 
FROM user_transactions 
GROUP BY UID;
'''
df = pd.read_sql_query(query, connection)
df

,uid,total_transactions
0,1,4
1,2,7
2,4,3
3,5,1
4,8,2
5,9,4


In [105]:
%%pydough

successful_transactions = transactions_sent.WHERE(sender_type == 0)
# Group users who have successful transactions and calculate the number of distinct transactions per user
transactions_by_users = Users.WHERE(HAS(successful_transactions))(
    uid,
    total_transactions= COUNT(successful_transactions)
)

pydough.to_df(transactions_by_users)

,uid,total_transactions
0,1,4
1,2,7
2,4,3
3,5,1
4,8,2
5,9,4


### What is the total session duration in seconds for each user between 2023-06-01 inclusive and 2023-06-08 exclusive? Return the user ID and their total duration as an integer sorted by total duration with the longest duration first

In [107]:
query = '''
WITH user_session_duration AS (
    SELECT 
        u.uid, 
        s.session_start_ts, 
        s.session_end_ts 
    FROM users AS u 
    JOIN user_sessions AS s 
        ON u.uid = s.user_id 
    WHERE 
        s.session_start_ts >= '2023-06-01' 
        AND s.session_end_ts < '2023-06-08'
)
SELECT 
    uid, 
    SUM(strftime('%s', session_end_ts) - strftime('%s', session_start_ts)) AS total_duration 
FROM user_session_duration 
GROUP BY uid 
ORDER BY total_duration DESC;

'''
df = pd.read_sql_query(query, connection)
df

,uid,total_duration
0,1,5018
1,6,4205
2,8,3640
3,2,3224
4,4,2736
5,5,2098
6,10,1843
7,9,1797
8,7,1518
9,3,1370


In [114]:
%%pydough

selected_sessions = sessions.WHERE(
    (session_start_ts >= "2023-06-01") &  
    (session_end_ts < "2023-06-08")  
)(duration=(SECOND(session_start_ts) - SECOND(session_end_ts))) #Pydough cannot convert dates to seconds directly, DATEDIFF

# Calculate the total session duration for each user and order by the total duration in descending order
total_session_duration = Users(
    uid,  
    total_duration=SUM(selected_sessions.duration) 
).ORDER_BY(total_duration.DESC())  

pydough.to_df(total_session_duration)

,uid,total_duration
0,3,10
1,9,3
2,5,2
3,11,0
4,6,-5
5,7,-18
6,4,-36
7,1,-38
8,8,-40
9,10,-43


### What is the total transaction amount for each coupon offered by merchant with ID 1? Return the coupon ID and total amount transacted with it.

In [116]:
query = '''
WITH merchant_coupon_usage AS (
    SELECT 
        c.cid, 
        t.amount 
    FROM 
        coupons AS c
    JOIN 
        wallet_transactions_daily AS t 
    ON 
        c.cid = t.coupon_id 
    WHERE 
        c.merchant_id = 1
)
SELECT 
    cid, 
    SUM(amount) AS total_discount 
FROM 
    merchant_coupon_usage 
GROUP BY 
    cid;
'''
df = pd.read_sql_query(query, connection)
df

,cid,total_discount
0,1,36.0


In [124]:
%%pydough
# Calculate the total session duration for each user and order by the total duration in descending order
total_session_duration = Coupons(
    cid,  
    total_discount=SUM(transaction_used_in.amount) 
).WHERE(merchant_id=='1') 

pydough.to_df(total_session_duration)

,cid,total_discount
0,1,36.0
